# Encoder-Decoder simples (Cap 2)

**Descrição:**
Este notebook apresenta uma implementação didática e simplificada do processo **encoder–decoder**, mostrando como textos são convertidos em representações numéricas (tokens) e, posteriormente, reconstruídos em texto. O foco está na preparação de dados textuais, tokenização e no fluxo conceitual de codificação e decodificação, conforme introduzido no Capítulo 2 do livro *Build a Large Language Model (From Scratch)*.

**Objetivo:**
Demonstrar, de forma prática e intuitiva, como funciona o pipeline básico de processamento de texto utilizado em modelos de linguagem:

* Construção de um corpus (vocabulário)
* Conversão de texto em tokens (encode)
* Representação do texto como vetores numéricos
* Reconstrução do texto a partir dos tokens (decode)

Ao final, o leitor deverá compreender como textos são transformados em dados numéricos que podem ser processados por modelos de linguagem.

**Funcionamento:**

![Exemplo de funcionamento de um encoder-decoder](../../imagens/02_encode_decode.png)


O funcionamento segue três etapas principais, conforme ilustrado na imagem de referência:

1. **Construção do corpus:**
   A partir de um conjunto de frases, criamos um vocabulário que associa cada token (palavra ou símbolo) a um identificador numérico.

2. **Encode (codificação):**
   Um novo texto é tokenizado e transformado em um vetor de inteiros, onde cada número representa um token do vocabulário.

3. **Decode (decodificação):**
   O vetor de tokens é convertido novamente em texto, demonstrando como a informação original pode ser recuperada a partir da representação numérica.

Esse fluxo simples representa a base de sistemas mais complexos usados em modelos de linguagem modernos, como transformers e LLMs.


## Implementação de um encoder-decoder simples

In [1]:
import re
from collections.abc import Iterable
from dataclasses import dataclass, field


@dataclass
class SimpleEncoderDecoder:
    """
    Implementação didática de um encoder-decoder baseado em vocabulário (corpus).

    A classe executa 3 etapas principais:
    1) gerar_corpus: constrói o vocabulário a partir de frases (corpus)
    2) encode: converte texto -> lista de token IDs
    3) decode: converte lista de token IDs -> texto

    Observações:
    - Tokenização simples via regex (separa palavras e pontuação básica)
    - Inclui prints com passo a passo, no estilo do exemplo do usuário
    - O vocabulário é ordenado alfabeticamente para reprodutibilidade (didático)
    """

    # Regex didática:
    # - Captura pontuação básica em um grupo separado: [,.!?;:]
    # - Captura espaços (\s) como separadores
    padrao: str = r"([,.!?;:]|\s)"

    # Estado interno (gerado a partir do corpus)
    vocab: dict[str, int] = field(default_factory=dict)  # token -> id
    inv_vocab: dict[int, str] = field(default_factory=dict)  # id -> token
    corpus_tokens: list[list[str]] = field(default_factory=list)  # tokens por frase

    def _tokenizar(self, texto: str, mostrar_passos: bool = True) -> list[str]:
        """
        Tokeniza um texto de forma simples (didática): separa pontuação e remove espaços/itens vazios.
        """
        if not isinstance(texto, str):
            raise TypeError("O parâmetro `texto` deve ser uma string (str).")

        if not texto.strip():
            raise ValueError("O parâmetro `texto` não pode ser vazio.")

        if mostrar_passos:
            print("=== TOKENIZAÇÃO: Texto original ===")
            print(texto)
            print()

            print("=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===")
        split_raw = re.split(self.padrao, texto)
        if mostrar_passos:
            print(split_raw)
            print()

            print("=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===")
        tokens = [p.strip() for p in split_raw if p.strip()]
        if mostrar_passos:
            print(tokens)
            print()

        return tokens

    def gerar_corpus(self, frases: Iterable[str]) -> None:
        """
        Constrói o vocabulário (corpus) a partir de uma lista/iterável de frases.

        Parâmetros:
        ----------
        frases : Iterable[str]
            Conjunto de frases para formar o corpus/vocabulário.

        Exceções:
        --------
        TypeError
            Se `frases` não for iterável ou contiver itens não-string.
        ValueError
            Se não houver tokens após a tokenização do corpus.
        """
        if frases is None:
            raise TypeError("O parâmetro `frases` não pode ser None.")

        frases = list(frases)
        if not frases:
            raise ValueError("O corpus precisa conter pelo menos 1 frase.")

        print("=== PASSO 1: Construção do corpus ===")
        print("Frases do corpus:")
        for i, f in enumerate(frases, start=1):
            print(f"  {i}. {f}")
        print()

        # Tokenizar cada frase (com passo a passo)
        self.corpus_tokens = []
        todos_tokens: list[str] = []

        for i, frase in enumerate(frases, start=1):
            print(f"--- Tokenizando frase {i} ---")
            tokens_frase = self._tokenizar(frase, mostrar_passos=True)
            self.corpus_tokens.append(tokens_frase)
            todos_tokens.extend(tokens_frase)

        if not todos_tokens:
            raise ValueError("Não foi possível gerar tokens a partir do corpus.")

        print("=== PASSO 2: Tokens coletados do corpus ===")
        print("Tokens por frase:")
        for i, toks in enumerate(self.corpus_tokens, start=1):
            print(f"  Frase {i}: {toks}")
        print()

        print("=== PASSO 3: Construção do vocabulário (token -> id) ===")
        tokens_unicos_ordenados = sorted(set(todos_tokens))
        self.vocab = {tok: idx for idx, tok in enumerate(tokens_unicos_ordenados)}
        self.inv_vocab = {idx: tok for tok, idx in self.vocab.items()}

        print("Tokens únicos (ordenados):")
        print(tokens_unicos_ordenados)
        print()
        print("Vocabulário (token -> id):")
        print(self.vocab)
        print()

    def encode(self, texto: str) -> list[int]:
        """
        Converte um texto em uma lista de token IDs com base no vocabulário do corpus.

        Parâmetros:
        ----------
        texto : str
            Texto a ser codificado.

        Retorno:
        -------
        list[int]
            Lista de token IDs.

        Exceções:
        --------
        ValueError
            Se o vocabulário ainda não foi gerado.
            Se houver token fora do vocabulário.
        """
        if not self.vocab:
            raise ValueError(
                "Vocabulário não encontrado. Execute `gerar_corpus()` antes de `encode()`."
            )

        print("=== ENCODE: Converter texto -> token IDs ===")
        tokens = self._tokenizar(texto, mostrar_passos=True)

        print("=== ENCODE: Checagem de tokens fora do vocabulário ===")
        desconhecidos = [t for t in tokens if t not in self.vocab]
        if desconhecidos:
            print("Tokens desconhecidos:", desconhecidos)
            raise ValueError(
                "Existem tokens que não estão no vocabulário do corpus. "
                "Para este exemplo didático, todos os tokens precisam existir no corpus."
            )
        print("Nenhum token desconhecido ✅")
        print()

        print("=== ENCODE: Conversão tokens -> IDs ===")
        token_ids = [self.vocab[t] for t in tokens]
        print("Tokens:", tokens)
        print("Token IDs:", token_ids)
        print()

        return token_ids

    def decode(self, token_ids: list[int]) -> str:
        """
        Converte uma lista de token IDs de volta para texto.

        Parâmetros:
        ----------
        token_ids : list[int]
            Lista de IDs.

        Retorno:
        -------
        str
            Texto reconstruído.

        Exceções:
        --------
        ValueError
            Se o vocabulário ainda não foi gerado.
            Se existir ID inválido.
        TypeError
            Se `token_ids` não for uma lista de inteiros.
        """
        if not self.inv_vocab:
            raise ValueError(
                "Vocabulário inverso não encontrado. Execute `gerar_corpus()` antes de `decode()`."
            )

        if not isinstance(token_ids, list) or any(
            not isinstance(i, int) for i in token_ids
        ):
            raise TypeError(
                "O parâmetro `token_ids` deve ser uma lista de inteiros (list[int])."
            )

        print("=== DECODE: Converter token IDs -> tokens ===")
        invalidos = [i for i in token_ids if i not in self.inv_vocab]
        if invalidos:
            print("IDs inválidos:", invalidos)
            raise ValueError("Existem IDs que não existem no vocabulário.")

        tokens = [self.inv_vocab[i] for i in token_ids]
        print("Token IDs:", token_ids)
        print("Tokens:", tokens)
        print()

        print("=== DECODE: Reconstrução do texto ===")
        # Regra simples: junta com espaço, mas remove espaço antes de pontuação
        texto = " ".join(tokens)
        texto = re.sub(r"\s+([,.!?;:])", r"\1", texto)

        print("Texto decodificado:", texto)
        print()
        return texto

### Geração do corpus

In [2]:
# =========================================================
# corpus com 4 frases (≈ 5 palavras) repetindo
# =========================================================
corpus = [
    "O gato sobe no tapete.",
    "O cachorro sobe na mesa.",
    "A aranha desce a parede.",
    "O gato desce da mesa.",
]

ed = SimpleEncoderDecoder()

# 01) Gerar corpus (vocabulário)
ed.gerar_corpus(corpus)

=== PASSO 1: Construção do corpus ===
Frases do corpus:
  1. O gato sobe no tapete.
  2. O cachorro sobe na mesa.
  3. A aranha desce a parede.
  4. O gato desce da mesa.

--- Tokenizando frase 1 ---
=== TOKENIZAÇÃO: Texto original ===
O gato sobe no tapete.

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['O', ' ', 'gato', ' ', 'sobe', ' ', 'no', ' ', 'tapete', '.', '']

=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===
['O', 'gato', 'sobe', 'no', 'tapete', '.']

--- Tokenizando frase 2 ---
=== TOKENIZAÇÃO: Texto original ===
O cachorro sobe na mesa.

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['O', ' ', 'cachorro', ' ', 'sobe', ' ', 'na', ' ', 'mesa', '.', '']

=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===
['O', 'cachorro', 'sobe', 'na', 'mesa', '.']

--- Tokenizando frase 3 ---
=== TOKENIZAÇÃO: Texto original ===
A aranha desce a parede.

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['A', ' '

### Exemplo de uso do encode (uma frase ainda não utilizada)

In [3]:
novo_texto = "A aranha sobe no gato."
token_ids = ed.encode(novo_texto)

print(">>> Resultado final do ENCODE")
print("Texto:", novo_texto)
print("Token IDs:", token_ids)

=== ENCODE: Converter texto -> token IDs ===
=== TOKENIZAÇÃO: Texto original ===
A aranha sobe no gato.

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['A', ' ', 'aranha', ' ', 'sobe', ' ', 'no', ' ', 'gato', '.', '']

=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===
['A', 'aranha', 'sobe', 'no', 'gato', '.']

=== ENCODE: Checagem de tokens fora do vocabulário ===
Nenhum token desconhecido ✅

=== ENCODE: Conversão tokens -> IDs ===
Tokens: ['A', 'aranha', 'sobe', 'no', 'gato', '.']
Token IDs: [1, 4, 13, 11, 8, 0]

>>> Resultado final do ENCODE
Texto: A aranha sobe no gato.
Token IDs: [1, 4, 13, 11, 8, 0]


### Exemplo de uso do decode (Uma frase ainda não utilizada)

In [4]:
token_ids = [2, 8, 13, 11, 5]
texto_decodificado = ed.decode(token_ids)

print(">>> Resultado final do DECODE")
print("Token IDs:", token_ids)
print("Texto:", texto_decodificado)

=== DECODE: Converter token IDs -> tokens ===
Token IDs: [2, 8, 13, 11, 5]
Tokens: ['O', 'gato', 'sobe', 'no', 'cachorro']

=== DECODE: Reconstrução do texto ===
Texto decodificado: O gato sobe no cachorro

>>> Resultado final do DECODE
Token IDs: [2, 8, 13, 11, 5]
Texto: O gato sobe no cachorro


## Propriedade das funções inversas:

Duas funções `a` e `b` são inversas se:

$$
b(a(x)) = x \quad \text{e} \quad a(b(y)) = y
$$

No caso do tokenizer:

$$
decode = encode^{-1}
$$



In [5]:
texto1 = "A aranha sobe no gato."
texto2 = ed.decode(ed.encode(novo_texto))

print("=" * 20)
print(texto1)
print(texto2)

=== ENCODE: Converter texto -> token IDs ===
=== TOKENIZAÇÃO: Texto original ===
A aranha sobe no gato.

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['A', ' ', 'aranha', ' ', 'sobe', ' ', 'no', ' ', 'gato', '.', '']

=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===
['A', 'aranha', 'sobe', 'no', 'gato', '.']

=== ENCODE: Checagem de tokens fora do vocabulário ===
Nenhum token desconhecido ✅

=== ENCODE: Conversão tokens -> IDs ===
Tokens: ['A', 'aranha', 'sobe', 'no', 'gato', '.']
Token IDs: [1, 4, 13, 11, 8, 0]

=== DECODE: Converter token IDs -> tokens ===
Token IDs: [1, 4, 13, 11, 8, 0]
Tokens: ['A', 'aranha', 'sobe', 'no', 'gato', '.']

=== DECODE: Reconstrução do texto ===
Texto decodificado: A aranha sobe no gato.

A aranha sobe no gato.
A aranha sobe no gato.


In [6]:
tokens1 = [1, 4, 13, 11, 8, 0]
tokens2 = ed.encode(ed.decode(tokens1))

print("=" * 20)
print(tokens1)
print(tokens2)

=== DECODE: Converter token IDs -> tokens ===
Token IDs: [1, 4, 13, 11, 8, 0]
Tokens: ['A', 'aranha', 'sobe', 'no', 'gato', '.']

=== DECODE: Reconstrução do texto ===
Texto decodificado: A aranha sobe no gato.

=== ENCODE: Converter texto -> token IDs ===
=== TOKENIZAÇÃO: Texto original ===
A aranha sobe no gato.

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['A', ' ', 'aranha', ' ', 'sobe', ' ', 'no', ' ', 'gato', '.', '']

=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===
['A', 'aranha', 'sobe', 'no', 'gato', '.']

=== ENCODE: Checagem de tokens fora do vocabulário ===
Nenhum token desconhecido ✅

=== ENCODE: Conversão tokens -> IDs ===
Tokens: ['A', 'aranha', 'sobe', 'no', 'gato', '.']
Token IDs: [1, 4, 13, 11, 8, 0]

[1, 4, 13, 11, 8, 0]
[1, 4, 13, 11, 8, 0]


## Falha ao 'encodar' palavras desconhecidas

In [7]:
texto = "O gato sobe na sopa"
ed.encode(texto)

=== ENCODE: Converter texto -> token IDs ===
=== TOKENIZAÇÃO: Texto original ===
O gato sobe na sopa

=== TOKENIZAÇÃO: Split bruto (inclui espaços e strings vazias) ===
['O', ' ', 'gato', ' ', 'sobe', ' ', 'na', ' ', 'sopa']

=== TOKENIZAÇÃO: Limpeza (remove espaços e itens vazios) ===
['O', 'gato', 'sobe', 'na', 'sopa']

=== ENCODE: Checagem de tokens fora do vocabulário ===
Tokens desconhecidos: ['sopa']


ValueError: Existem tokens que não estão no vocabulário do corpus. Para este exemplo didático, todos os tokens precisam existir no corpus.